# Netflix Campaign Visual Generator (DALL·E + Gradio)

This notebook demonstrates a lightweight **web-based creative platform** that turns **text prompts** into **bespoke banner/poster-ready images** using **OpenAI's DALL·E image generation** and a **Gradio** web interface.

- Imports essential libraries (**gradio**, **requests**, **PIL**, **io.BytesIO**, **openai**)
- Implements `generate_image(prompt)` to generate an image from a text prompt
- Creates a **Gradio** UI with textboxes, choice of number of images to generate, and an image gallery
- Launches the app for interactive use

## Verify environment


In [1]:
# Verify environment
import os
import socket
print({k:v for k,v in os.environ.items() if "JUPYTER" in k or "PROXY" in k or "PORT" in k})
# Jupyter typically sets one of these in managed platforms:
base = os.environ.get("JUPYTERHUB_BASE_URL") or os.environ.get("VOC_JUPYTER_KERNEL_BASE_URL") or "/"
print("Detected base path:", base)
print("Try these paths appended to your notebook domain:")
print(f"{base.rstrip('/')}/proxy/7864/")
print(f"{base.rstrip('/')}/7864/")
print("Hostname:", socket.gethostname())
print("JUPYTERHUB_SERVICE_PREFIX =", os.environ.get("JUPYTERHUB_SERVICE_PREFIX"))
print("BASE_URL =", os.environ.get("JUPYTERHUB_BASE_URL"))

{'VOC_JUPYTER_ROOT_DIR': '/voc', 'VOC_JUPYTER_IDE_TYPE': 'lab', 'VOC_JUPYTER_START_VOCKERNEL': 'false', 'VOC_JUPYTER_KERNEL_BASE_URL': '/', 'VOC_PROXY_ROUTE_jupyter_8888': '/', 'VOC_JUPYTER_PREF_DIR_STR': '--preferred-dir=/voc/work', 'VOC_JUPYTER_PREF_DIR': '/home/labsuser', 'VOC_JUPYTER_START_JUPYTER': 'true', 'VOC_JUPYTER_USERNAME': 'labsuser'}
Detected base path: /
Try these paths appended to your notebook domain:
/proxy/7864/
/7864/
Hostname: jupyter
JUPYTERHUB_SERVICE_PREFIX = None
BASE_URL = None


## Install Gradio and its dependencies

In [2]:
# --- Step 1: Install required Gradio runtime dependencies (idempotent) ---
!pip install -U --no-deps \
  groovy==0.1.2 \
  brotli==1.1.0 \
  gradio-client==2.0.3 \
  "huggingface-hub>=0.33.5,<2.0" \
  "starlette>=0.40.0,<1.0" \
  "fastapi>=0.115.2,<1.0"

# --- Step 2: Conditionally install / upgrade Gradio itself ---
import subprocess
import re
from packaging.version import Version, InvalidVersion

def get_installed_gradio_version():
    try:
        import gradio as gr
        return gr.__version__
    except Exception:
        return None

# Query pip for available versions of gradio
result = subprocess.run(
    ["pip", "index", "versions", "gradio"],
    capture_output=True,
    text=True,
)

versions = re.findall(r"\b\d+\.\d+\.\d+\b", result.stdout)
if not versions:
    raise RuntimeError("Could not detect available Gradio versions.")

latest_version = versions[0]  # pip lists newest first
installed_version = get_installed_gradio_version()

print("Installed Gradio version:", installed_version)
print("Latest available Gradio version:", latest_version)

needs_install = (
    installed_version is None
    or Version(installed_version) < Version(latest_version)
)

if needs_install:
    print("Installing / upgrading Gradio to:", latest_version)
    subprocess.run(
        ["pip", "install", "-U", "--no-deps", f"gradio=={latest_version}"],
        check=True,
    )
    print("Gradio installed:", latest_version)
    print("⚠️ Restart the kernel before importing gradio")
else:
    print("Gradio is already up to date. No action needed.")

!pip install pillow requests openai safehttpx annotated-doc
!pip install -U "typing_extensions>=4.10"
import typing_extensions as te
print("typing_extensions file:", te.__file__)
print("Has TypeIs:", hasattr(te, "TypeIs"))
print("Has __version__:", hasattr(te, "__version__"))

Defaulting to user installation because normal site-packages is not writeable
Installed Gradio version: 6.3.0
Latest available Gradio version: 6.3.0
Gradio is already up to date. No action needed.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
typing_extensions file: /voc/work/.local/lib/python3.10/site-packages/typing_extensions.py
Has TypeIs: True
Has __version__: False


## Import packages

In [3]:
import os, openai, json, requests
import gradio as gr
print("Gradio:", gr.__version__)
from PIL import Image
from io import BytesIO

Gradio: 6.3.0


## Set LLM API key

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise EnvironmentError("OPENAI_API_KEY is not set.")

## Build LLM prompt

In [5]:
def build_prompt(title: str, concept: str) -> str:
    title = (title or "").strip()
    concept = (concept or "").strip()

    if not title:
        raise gr.Error("Please enter the show/movie name.")
    if not concept:
        raise gr.Error("Please enter the concept/creative direction.")

    # Prompt template tuned for Netflix-style key art ideation
    return f"""
Create premium Netflix-style key art for the title: "{title}".

Creative concept:
{concept}

Design notes:
- cinematic lighting, high contrast, strong focal subject
- clean composition with negative space for marketing copy
- polished, high-end editorial finish
- avoid embedded text or logos; focus on visual concept only
""".strip()

## Define image generation function

`generate_image`:
1. Calls OpenAI Images API (DALL·E) using the prompt
2. Retrieves the returned image URLs via `requests`
3. Loads images into a `PIL.Image` objects and returns them to Gradio


In [6]:
def generate_images(title: str, concept: str, batch_size: int):
    prompt = build_prompt(title, concept)

    batch_size = int(batch_size)
    if batch_size < 1 or batch_size > 8:
        raise gr.Error("Batch size must be between 1 and 8.")

    images = []
    for _ in range(batch_size):
        response = openai.Image.create(
            prompt=prompt,
            n=1,
            size="1024x1024",
        )

        if "data" not in response:
            raise gr.Error(f"OpenAI image generation error: {response.get('error', response)}")

        image_url = response["data"][0]["url"]
        img_bytes = requests.get(image_url, timeout=60).content
        img = Image.open(BytesIO(img_bytes)).convert("RGB")
        images.append(img)

    return images

## Define Gradio interface

This UI exposes:
- **Input:** Textbox for creative prompt
- **Output:** Image preview

You can extend this later with:
- Negative prompting / style controls
- Batch generation and gallery
- Simple brand guardrails for Netflix campaign consistency


In [7]:
demo = gr.Interface(
    fn=generate_images,
    inputs=[
        gr.Textbox(
            label="Show / Movie Name",
            placeholder='Example: "Eclipse Protocol"',
            lines=1,
        ),
        gr.Textbox(
            label="Concept / Creative Direction",
            placeholder="Example: A neo-noir conspiracy thriller in rainy Seoul, neon reflections, lone protagonist, ominous surveillance motifs.",
            lines=5,
        ),
        gr.Slider(
            minimum=1,
            maximum=8,
            step=1,
            value=4,
            label="Batch size (images per run)"
        ),
    ],
    outputs=gr.Gallery(
        label="Generated concepts",
        columns=4,
        rows=2,
        height="auto",
        show_label=True,
    ),
    title="Netflix Campaign Visual Generator (Title + Concept → Batch Gallery)",
    description="Enter the title and concept. The app combines them into a single prompt and generates multiple key-art concepts.",
)

## Launch Gradio app

In [8]:
CUSTOM_CSS = """
#root { max-width: 95vw; }
.gradio-container { min-height: 92vh; }
"""

In [9]:
# In notebooks, set share=True if you want a public link.
demo.launch(share=True, css=CUSTOM_CSS)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://b4a0c8673d3eb5e224.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
